In [1]:
#!pip install datasets
#!pip install scikit-learn
#!pip install transformers

In [2]:
import datasets
import torch

In [96]:
import datasets
dataset = datasets.load_dataset("harpomaxx/dga-detection")
dataset['train'].shuffle()['label'][1:100]

Found cached dataset dga-detection (/root/.cache/huggingface/datasets/harpomaxx___dga-detection/default/0.0.0/4bf750f8e009387bc45fc3d21c897b3c951c4d8cdf18a69d513e481198d1beb0)


  0%|          | 0/3 [00:00<?, ?it/s]

['normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'dga.tinba',
 'dga.post',
 'dga.necurs',
 'dga.geodo',
 'dga.post',
 'dga.symmi',
 'dga.banjori',
 'dga.qakbot',
 'normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'dga.pykspa',
 'dga.banjori',
 'normal.alexa',
 'dga.banjori',
 'normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'dga.pykspa',
 'normal.alexa',
 'dga.post',
 'dga.post',
 'dga.tinba',
 'dga.ramnit',
 'dga.cryptolocker',
 'dga.qakbot',
 'normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'normal.alexa',
 'dga.dyre',
 'dga.dyre',
 'dga.banjori',
 'dga.kraken',
 'dga.shiotob/urlzone/bebloh',
 'dga.banjori',
 'dga.kraken',
 'normal.alexa',
 'dga.tinba',
 'dga.banjori',
 'dga.banjori',
 'normal.alexa',
 'dga.rovnix',
 'normal.alexa',
 'normal.alexa',
 'dga.post',
 'dga.qakbot',
 'normal.alexa',
 'normal.alexa',
 'dga.banjori',
 'normal.alexa',
 'normal.alexa',
 'dga.banjori',
 'dga.ranbyus',
 'dga.post',
 'normal.alexa',
 'dga

In [4]:
dataset.keys()
dataset['train'][0]

{'domain': '0-1.ru', 'label': 'normal.alexa', 'class': 0}

In [5]:
dataset['train'].select(range(0,1024))

Dataset({
    features: ['domain', 'label', 'class'],
    num_rows: 1024
})

In [6]:
model_checkpoint = "distilgpt2"
model_checkpoint = "gpt2"

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast = True )

**Note:**
When you're using a new tokenizer with a different vocabulary size, the original token embeddings may not match the dimensions of the new tokenizer's vocabulary. In order to address this mismatch, you need to resize the token embeddings to match the new tokenizer's vocabulary size.

The padding token is used to pad sequences to a fixed length during tokenization, especially when working with batched data. It ensures that all sequences have the same length, which is necessary for efficient processing in deep learning models. By default, Hugging Face tokenizers use a special token called [PAD] as the padding token.

In some cases, you may want to change the padding token to a different token in the tokenizer's vocabulary. In the line tokenizer.pad_token = tokenizer.eos_token, the padding token is set to be the same as the end-of-sequence (EOS) token.

By making the padding token the same as the EOS token, you are essentially indicating that sequences should be padded with the EOS token. This means that during tokenization, when a sequence is shorter than the maximum length, it will be padded with EOS tokens until it reaches the maximum length.

In [8]:
# default to left padding
# tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token

In [9]:
tokenizer("hello people")

{'input_ids': [31373, 661], 'attention_mask': [1, 1]}

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["domain"])

We remove the `domain` and `label` columns an apply the tokenization function to all the datasets

In [11]:
tokenized_datasets = dataset.map(tokenize_function, 
                                 batched=True, 
                                 num_proc=8, 
                                 remove_columns=["domain","label"],
                                 )

Map (num_proc=8):   0%|          | 0/2042947 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/583699 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/291850 [00:00<?, ? examples/s]

Now create a batch of examples using DataCollatorWithPadding. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
dataset["train"][5]
tokenized_datasets["train"][1]

{'class': 0,
 'input_ids': [15, 12, 21, 13, 785],
 'attention_mask': [1, 1, 1, 1, 1]}

Rename  the `class` column to `label`

In [14]:
tokenized_datasets = tokenized_datasets.rename_column("class","labels")

In [15]:
encoded_inputs = tokenizer(dataset['train'][1:100]['domain'], padding=True, truncation=True, return_tensors='pt')
batch = data_collator(encoded_inputs)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
batch

{'input_ids': tensor([[   15,    12,    21,  ..., 50256, 50256, 50256],
        [   15,    12,  1899,  ..., 50256, 50256, 50256],
        [   15,    12,  1899,  ..., 50256, 50256, 50256],
        ...,
        [25816, 35634,  2189,  ..., 50256, 50256, 50256],
        [25816, 35634,  2189,  ..., 50256, 50256, 50256],
        [25816,  2777,    88,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [17]:
dataset['train'][1]

{'domain': '0-6.com', 'label': 'normal.alexa', 'class': 0}

In [18]:
from transformers import GPT2ForSequenceClassification
model = GPT2ForSequenceClassification.from_pretrained(model_checkpoint,num_labels=2)
# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
import evaluate

metric = evaluate.load("f1")

In [20]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [21]:
from transformers import Trainer, TrainingArguments

In [36]:
training_args = TrainingArguments(
    output_dir="gpt2-dga-detector",
    learning_rate=2e-5,
    optim="adamw_torch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_steps = 10,
    save_total_limit = 3,
    overwrite_output_dir = True
)

In [87]:
len(dataset['validation']['domain'])

291850

In [24]:
tokenized_datasets['train'].shuffle(seed=42).select(range(0,2048))

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2048
})

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'].shuffle(seed=42).select(range(0,8192)),
    eval_dataset=tokenized_datasets['test'].shuffle(seed=42).select(range(0,1024)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/harpomaxx___dga-detection/default/0.0.0/4bf750f8e009387bc45fc3d21c897b3c951c4d8cdf18a69d513e481198d1beb0/cache-1a8c954236c7abf8.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/harpomaxx___dga-detection/default/0.0.0/4bf750f8e009387bc45fc3d21c897b3c951c4d8cdf18a69d513e481198d1beb0/cache-6f30e67f5447515f.arrow


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=4096, training_loss=0.008644755998890585, metrics={'train_runtime': 287.8638, 'train_samples_per_second': 227.663, 'train_steps_per_second': 14.229, 'total_flos': 688778234560512.0, 'train_loss': 0.008644755998890585, 'epoch': 8.0})

In [27]:
metrics=trainer.evaluate()
print(metrics)

{'eval_loss': 0.5324081182479858, 'eval_f1': 0.9304500292226767, 'eval_runtime': 0.7309, 'eval_samples_per_second': 1400.999, 'eval_steps_per_second': 87.562, 'epoch': 8.0}


In [28]:
model.save_pretrained("./CEPH/gpt2-dga-detector")
tokenizer.save_pretrained("./CEPH/gpt2-dga-detector")

('./CEPH/gpt2-dga-detector/tokenizer_config.json',
 './CEPH/gpt2-dga-detector/special_tokens_map.json',
 './CEPH/gpt2-dga-detector/vocab.json',
 './CEPH/gpt2-dga-detector/merges.txt',
 './CEPH/gpt2-dga-detector/added_tokens.json',
 './CEPH/gpt2-dga-detector/tokenizer.json')

In [97]:
from transformers import pipeline

#classifier = pipeline("zero-shot-classification")
#classifier(
#    "This is a course about the Transformers library",
#    candidate_labels=["education", "politics", "business"],
#)
#classifier = pipeline("zero-shot-classification", model="gpt2",device=0)

classifier = pipeline("text-classification", model="./CEPH/gpt2-dga-detector/",device=0,batch_size=2)


In [16]:
import numpy as np

In [17]:
np.unique(dataset['train']['label'])

array(['dga.bamital', 'dga.banjori', 'dga.bedep', 'dga.beebone',
       'dga.chinad', 'dga.conficker', 'dga.corebot', 'dga.cryptolocker',
       'dga.cryptowall', 'dga.dircrypt', 'dga.dyre', 'dga.fobber',
       'dga.gameover', 'dga.geodo', 'dga.hesperbot', 'dga.kraken',
       'dga.locky', 'dga.matsnu', 'dga.murofet', 'dga.necurs',
       'dga.nymaim', 'dga.p2p', 'dga.padcrypt', 'dga.post',
       'dga.proslikefan', 'dga.pushdo', 'dga.pykspa', 'dga.pykspa_v1',
       'dga.pykspa_v2_fake', 'dga.pykspa_v2_real', 'dga.qadars',
       'dga.qakbot', 'dga.ramdo', 'dga.ramnit', 'dga.ranbyus',
       'dga.rovnix', 'dga.shifu', 'dga.shiotob/urlzone/bebloh',
       'dga.simda', 'dga.sisron', 'dga.suppobox', 'dga.symmi',
       'dga.tempedreve', 'dga.tinba', 'dga.vawtrak', 'dga.virut',
       'dga.volatile', 'normal.alexa'], dtype='<U26')

In [91]:
np.unique(dataset['validation']['label'],return_counts= True)

(array(['dga.conficker', 'dga.cryptolocker', 'dga.matsnu', 'dga.pushdo',
        'dga.ramdo', 'dga.tinba', 'dga.zeus', 'normal.bambenek'],
       dtype='<U16'),
 array([ 17905,  13576,  23275,  33926, 100000,  99000,   1000,   3168]))

In [93]:
236960/(236960+49884)

0.8260936258035727

In [95]:
1-(2701/(2701+454))

0.14389857369255155

In [1]:
t= dataset['validation'][1:100]
t

NameError: name 'dataset' is not defined

In [113]:
classifier = pipeline("text-classification", model="./CEPH/gpt2-dga-detector/",device=0,batch_size=1024)

In [116]:
predictions = classifier(dataset['validation']['domain'])
predictions

[{'label': 'LABEL_0', 'score': 0.9999750852584839},
 {'label': 'LABEL_1', 'score': 0.9999996423721313},
 {'label': 'LABEL_1', 'score': 0.999976396560669},
 {'label': 'LABEL_1', 'score': 0.999648928642273},
 {'label': 'LABEL_0', 'score': 0.9999771118164062},
 {'label': 'LABEL_1', 'score': 0.9999997615814209},
 {'label': 'LABEL_1', 'score': 0.9999990463256836},
 {'label': 'LABEL_1', 'score': 0.9997816681861877},
 {'label': 'LABEL_0', 'score': 0.9999552965164185},
 {'label': 'LABEL_1', 'score': 0.9999994039535522},
 {'label': 'LABEL_0', 'score': 0.9987151622772217},
 {'label': 'LABEL_0', 'score': 0.9910101890563965},
 {'label': 'LABEL_0', 'score': 0.9999134540557861},
 {'label': 'LABEL_1', 'score': 0.999998927116394},
 {'label': 'LABEL_1', 'score': 0.9997219443321228},
 {'label': 'LABEL_1', 'score': 0.956656277179718},
 {'label': 'LABEL_0', 'score': 0.9999903440475464},
 {'label': 'LABEL_1', 'score': 0.9999995231628418},
 {'label': 'LABEL_0', 'score': 0.9421871900558472},
 {'label': 'LABE

In [117]:
predicted_labels = [pred['label'] for pred in predictions]

predicted_labels =np.array(predicted_labels)
predicted_labels = np.where(predicted_labels == 'LABEL_0', 0, 1)
predicted_labels

array([0, 1, 1, ..., 0, 0, 0])

In [119]:
#predicted_labels = np.array(predictions['label'][1:100])
true_labels = np.array(dataset['validation']['class'])


In [120]:
true_labels

array([1, 1, 1, ..., 0, 0, 0])

In [126]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [122]:
cm = confusion_matrix(true_labels, predicted_labels)
cm

array([[  2711,    457],
       [ 50231, 238451]])

In [127]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)
# Calculate F1 score
recall = recall_score(true_labels, predicted_labels)
# Calculate F1 score
precision = precision_score(true_labels, predicted_labels)

In [128]:
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall Score: {recall}")
print(f"Precision Score: {precision}")

Accuracy: 0.8263217406201816
F1 Score: 0.9039253966147954
Recall Score: 0.8259988499456149
Precision Score: 0.9980871297738041


In [129]:
import pandas as pd

In [130]:
pd.DataFrame(predictions).to_csv("predictions.csv")

,label,score
0,LABEL_0,0.999975
1,LABEL_1,1.000000
2,LABEL_1,0.999976
3,LABEL_1,0.999649
4,LABEL_0,0.999977
...,...,...
291845,LABEL_0,1.000000
291846,LABEL_0,0.999993
291847,LABEL_0,0.999993
291848,LABEL_0,0.998776
